# Training Script

In [3]:
import os, object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

2023-03-07 16:24:45.177884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
modelName = "640x640_model"
pretrainedModel = "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
pretrainedModelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz"

# Script sourced from Tensorflow: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/tensorflow-1.14/training.html
tfRecordScript = "generate_tfrecord.py"
labelMap = "label_map.pbtxt"

In [5]:
filePaths = {
    "WORKSPACE": os.path.join("TF", "workspace"),
    "SCRIPTS": os.path.join("TF", "scripts"),
    "APIMODEL": os.path.join("TF", "models"),
    "ANNOTATIONS": os.path.join("TF", "workspace", "annotations"),
    "IMAGES": os.path.join("TF", "workspace", "images"),
    "PRETRAINED_MODELS": os.path.join("TF", "workspace", "pretrained_models"),
}

In [9]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', modelName, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(filePaths['SCRIPTS'], tfRecordScript),
    'LABELMAP': os.path.join(filePaths['ANNOTATIONS'], labelMap)
}

In [ ]:
for path in filePaths.values():
    !mkdir -p {pat}